In [1]:
#load packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/relea

In [2]:
#load training data and labels for data that includes 
#MMETSP transcriptomes with high contamination and 
#low sequence abundance
train = pd.read_csv('trainingData_withContam_withLowSeqs.csv')

#get just the TPM values not the MMETSP entry IDs and trophic mode labels
trainData = train.iloc[:, 2:]

#load feature Pfams from previous version of model (Lambert et al)
features = pd.read_csv('Extracted_Pfams_lambert.csv')

In [3]:
#extract pfam column from features dataframe 
#without period and decimals after pfam id
features = features['Pfam'].str.split('.').str[0]

In [4]:
#need to encode trophic labels as numbers (0,1,2)
le = LabelEncoder()

In [5]:
#get just data for the feature Pfams from training data
trainData = trainData[features]

In [6]:
#assign feature matrix and target vector
X, y = trainData, le.fit_transform(train['Trophic mode'])
#X: Feature matrix (independent variables) from the DataFrame trainData
#y: Target vector (dependent variable), where the 'Trophic mode' column is label-encoded using `LabelEncoder`.

#initialize a MinMaxScaler instance
scaler = MinMaxScaler()
#minMaxScaler scales features to a specified range, typically [0, 1], which can improve the performance of machine learning models

#scale feature matrix
X = scaler.fit_transform(X)

In [7]:
#split the dataset into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(
    X,           # Features (input variables)
    y,           # Target variable (output/labels)
    test_size=0.4, # Proportion of the dataset to include in the test split (40%)
    random_state=0 # Random seed for reproducibility of the split
)

In [8]:
np.random.seed(7)

#model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
clf = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)

#define a custom scoring function for F1 score
f1_scorer = make_scorer(f1_score, average='weighted')

#create a StratifiedShuffleSplit cross-validator
kf = StratifiedShuffleSplit(n_splits=6, random_state=7)

#perform cross-validation and store the F1 scores
f1_scores = cross_val_score(clf, X, y, cv=kf, scoring=f1_scorer)

#save F1 scores, mean, and standard error to csv file
data = {'F1_Scores': f1_scores}

df = pd.DataFrame(data)

#calculate mean and standard error of F1 scores
mean_f1 = data['F1_Scores'].mean()
std_error_f1 = data['F1_Scores'].std(ddof=1) / np.sqrt(len(data['F1_Scores']))

#create a new DataFrame with the results
df = pd.DataFrame({
    'Model': ['Previous model Lambert et al.2022 (xgboost model, xgboost and random forest feature Pfams, training data with contamination and low sequence abundance, not updated software)'],
    'Mean_F1_Score': [mean_f1],
    'Std_Error_F1_Score': [std_error_f1]
})

#save overall F1 score to CSV file
df.to_csv('lambert_model_overall_f1_score.csv', index=False)

/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

In [9]:
np.random.seed(7)

#model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
clf = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)

#define a custom scoring function for F1 score
def f1_scorer(y_true, y_pred):
    return f1_score(y_true, y_pred, average=None)

#create a StratifiedShuffleSplit cross-validator
kf = StratifiedShuffleSplit(n_splits=6, random_state=7)

#initialize an empty list to store F1 scores for each class
all_f1_scores = []

#perform cross-validation and store the F1 scores for each fold
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    fold_f1_scores = f1_scorer(y_test, y_pred)
    all_f1_scores.append(fold_f1_scores)

#convert the list of arrays to a numpy array
f1_scores = np.array(all_f1_scores)

#save the F1 scores for each trophic mode separately
trophic_names = [f'Trophic_{i}' for i in range(f1_scores.shape[1])]
data = {trophic_name: f1_scores[:, i] for i, trophic_name in enumerate(trophic_names)}

#prepare a list to store results
results = []

#loop through the data dictionary and calculate mean and standard error for each class
for class_name, f1_scores in data.items():
    mean_f1 = np.mean(f1_scores)
    std_error_f1 = np.std(f1_scores, ddof=1) / np.sqrt(len(f1_scores))  # Standard error
    results.append({
        'Model': 'Previous model Lambert et al.2022 (xgboost model, xgboost and random forest feature Pfams, training data with contamination and low sequence abundance, not updated software)',
        'Trophic mode': class_name,
        'Mean_F1_Score': mean_f1,
        'Std_Error_F1_Score': std_error_f1
    })

#create a DataFrame from the results
df = pd.DataFrame(results)

# Replace Trophic values with desired labels
df['Trophic mode'] = df['Trophic mode'].replace({'Trophic_0': 'Het', 'Trophic_1': 'Mix', 'Trophic_2': 'Phot'})

#save overall F1 score by trophic mode to CSV file
df.to_csv('lambert_model_overall_f1_score_byClass.csv', index=False)

/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

In [11]:
#generate confusion matrix summed across folds of cross-validation

cumulative_cm = np.zeros((3, 3), dtype=int)  # 3 classes

# Redefine and rerun the loop if needed
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    cumulative_cm += cm

class_names = ['Het', 'Mix', 'Phot']
cm_df = pd.DataFrame(cumulative_cm, index=class_names, columns=class_names)
cm_df.to_csv('lambert_model_cumulative_confusion_matrix.csv')

/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre